In [ ]:
import pandas as pd
from model_utils.utils_model import Predictor
from model_utils.util_global_struct import process_bb_old_to_new

In [ ]:
import numpy as np
import torch
from model_utils.utils_model import DataEncoder

In [ ]:
from scipy.special import softmax

In [ ]:
import plotly.express as px

In [ ]:
df = pd.read_pickle('../2021_03_16/data/human_transcriptome_segment_high_mfe_freq_testing_len64_100.pkl.gz')

In [ ]:
model_path = 's1_training/result/run_7/model_ckpt_ep_17.pth'  # best model

predictor = Predictor(model_ckpt=model_path,
                     num_filters=[32, 32, 64, 64, 64, 128, 128],
                     filter_width=[9, 9, 9, 9, 9, 9, 9],
                     dropout=0.0)


In [ ]:
idx = 99
seq = df.iloc[idx]['seq']
one_idx = df.iloc[idx]['one_idx']
bounding_boxes = df.iloc[idx]['bounding_boxes']
df_target = process_bb_old_to_new(bounding_boxes)

In [ ]:
seq

In [ ]:
df_target

In [ ]:
de = DataEncoder(seq)
yp = predictor.model(torch.tensor(de.x_torch))
yp = {k: v.detach().cpu().numpy()[0, :, :, :] for k, v in yp.items()}

In [ ]:
yp.keys()

In [ ]:
stem_on = yp['stem_on'][0, :, :]
# hard-mask lower triangular part
stem_on[np.tril_indices(len(seq))] = 0
fig = px.imshow(stem_on)
fig.update_layout(title='stem_on')

In [ ]:
yp['stem_sl_size'][0, 29:29+4, 55+1-4:55+1]

In [ ]:
# px.imshow(yp['stem_sl_size'][0, 29-2:29+4+2, 55+1-4-2:55+1+2])
# px.imshow(yp['stem_sl_size'][0, :, :])

In [ ]:
def make_pixel_shift_plot(yp, pred_key, x_tr, y_tr,  bb_size, offset, direction):
    df_plot = pd.DataFrame()
    for offset in range(-offset, bb_size+offset):
        if direction == 'x':
            val = softmax(yp[pred_key][:, x_tr+offset, y_tr])
            df_plot['location {}'.format(x_tr+offset)] = val
        elif direction == 'y':
            val = softmax(yp[pred_key][:, x_tr, y_tr-offset])
            df_plot['location {}'.format(y_tr-offset)] = val

    fig = px.line(df_plot)
    return fig

In [ ]:
# x_tr, y_tr = 29, 55
# print(yp['stem_on'][0, x_tr, y_tr])
# bb_size = 4
# direction = 'y'
# pred_key = 'stem_location_y'

pred_key = 'stem_location_y'
fig = make_pixel_shift_plot(yp, pred_key=pred_key,
                            x_tr=29, y_tr=55,  
                            bb_size=4, offset=2, direction='y')
fig.update_layout(title=pred_key, height=500, width=600)


In [ ]:
pred_key = 'stem_location_x'
fig = make_pixel_shift_plot(yp, pred_key=pred_key,
                            x_tr=29, y_tr=55,  
                            bb_size=4, offset=2, direction='x')
fig.update_layout(title=pred_key, height=500, width=600)


In [ ]:
iloop_on = yp['iloop_on'][0, :, :]
# hard-mask lower triangular part
iloop_on[np.tril_indices(len(seq))] = 0
fig = px.imshow(iloop_on)
fig.update_layout(title='iloop_on')

In [ ]:
pred_key = 'iloop_location_y'
fig = make_pixel_shift_plot(yp, pred_key=pred_key,
                            x_tr=18, y_tr=36,  
                            bb_size=4,  offset=2, direction='y')
fig.update_layout(title=pred_key, height=500, width=600)

In [ ]:
pred_key = 'iloop_location_x'
fig = make_pixel_shift_plot(yp, pred_key=pred_key,
                            x_tr=18, y_tr=36,  
                            bb_size=6,  offset=2, direction='x')
fig.update_layout(title=pred_key, height=500, width=600)

In [ ]:
iloop_on = yp['hloop_on'][0, :, :]
# hard-mask lower triangular part
iloop_on[np.tril_indices(len(seq))] = 0
fig = px.imshow(iloop_on)
fig.update_layout(title='hloop_on')

In [ ]:
pred_key = 'hloop_location_y'
fig = make_pixel_shift_plot(yp, pred_key=pred_key,
                            x_tr=48, y_tr=52,  
                            bb_size=4, offset=2, direction='y')
fig.update_layout(title=pred_key, height=500, width=600)

In [ ]:
pred_key = 'hloop_location_x'
fig = make_pixel_shift_plot(yp, pred_key=pred_key,
                            x_tr=48, y_tr=52,  
                            bb_size=4, offset=2, direction='x')
fig.update_layout(title=pred_key, height=500, width=600)

In [ ]:
yp.keys()

In [ ]:

pred_key = 'hloop_sm_size'
fig = make_pixel_shift_plot(yp, pred_key=pred_key,
                            x_tr=48, y_tr=52,  
                            bb_size=4, offset=2, direction='y')
fig.update_layout(title=pred_key, height=500, width=600)